In [ ]:
import re
import random
import numpy as np
import pandas as pd

import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt

from collections import Counter

from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyClassifier
from sklearn.metrics import (
    classification_report,
    confusion_matrix,
    precision_score,
    recall_score,
    f1_score,
    classification_report,
    confusion_matrix,
    precision_recall_curve,
    average_precision_score
)
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader, Dataset

from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    get_linear_schedule_with_warmup
)

c:\Users\jean-\anaconda3\envs\llm\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Paramètres

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

RANDOM_STATE = 42
random.seed(RANDOM_STATE)
np.random.seed(RANDOM_STATE)

pd.set_option("display.max_colwidth", 120)

# Import dataset

In [3]:
DATA_PATH = "../data/outputs/spam_cleaned.csv"

df = pd.read_csv(DATA_PATH, encoding="latin-1")
df.head()

,Unnamed: 0,label,text,y
0,0,ham,"Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...",0
1,1,ham,Ok lar... Joking wif u oni...,0
2,2,spam,Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std t...,1
3,3,ham,U dun say so early hor... U c already then say...,0
4,4,ham,"Nah I don't think he goes to usf, he lives around here though",0


In [4]:
df.shape

(5572, 4)

# Baseline

## Split stratifié (train/test)

In [5]:
X = df["text"]
y = df["y"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    stratify=y,
    random_state=RANDOM_STATE
)

print("Train size:", X_train.shape[0], "| Test size:", X_test.shape[0])
print("Spam rate train:", y_train.mean().round(4), "| Spam rate test:", y_test.mean().round(4))


Train size: 4457 | Test size: 1115
Spam rate train: 0.1342 | Spam rate test: 0.1336


## Baseline 0 - Naïf "stratified"

In [6]:
Xtr_dummy = np.zeros((len(y_train), 1))
Xte_dummy = np.zeros((len(y_test), 1))

dummy_strat = DummyClassifier(strategy="stratified", random_state=RANDOM_STATE)
dummy_strat.fit(Xtr_dummy, y_train)

y_pred_strat = dummy_strat.predict(Xte_dummy)
y_proba_strat = dummy_strat.predict_proba(Xte_dummy)[:, 1]

### Métriques et Confusion Matrix

In [7]:
print("Baseline 0 - Naïf (stratified random)")
print(classification_report(y_test, y_pred_strat, target_names=["ham(0)", "spam(1)"]))
print("Confusion matrix:\n", confusion_matrix(y_test, y_pred_strat))

Baseline 0 - Naïf (stratified random)
              precision    recall  f1-score   support

      ham(0)       0.86      0.85      0.85       966
     spam(1)       0.10      0.11      0.10       149

    accuracy                           0.75      1115
   macro avg       0.48      0.48      0.48      1115
weighted avg       0.76      0.75      0.75      1115

Confusion matrix:
 [[817 149]
 [133  16]]


In [ ]:
cm = confusion_matrix(y_test, y_pred_strat)
cm_df = pd.DataFrame(
    cm,
    index=["True ham (0)", "True spam (1)"],
    columns=["Pred ham (0)", "Pred spam (1)"]
)

fig = px.imshow(
    cm_df,
    text_auto=True,
    aspect="auto",
    title="Confusion Matrix : Naïf (stratified random)"
)
fig.update_layout(xaxis_title="Predicted label", yaxis_title="True label")
fig.show()

## Baseline TF-IDF + Regression logistique

In [9]:
baseline_clf = Pipeline(steps=[
    ("tfidf", TfidfVectorizer(
        lowercase=True,
        strip_accents="unicode",
        ngram_range=(1, 2),
        min_df=2,
        max_df=0.95
    )),
    ("logreg", LogisticRegression(
        max_iter=2000,
        class_weight="balanced", # important vu l'imbalance
        random_state=RANDOM_STATE
    ))
])

baseline_clf.fit(X_train, y_train)

,steps,"[('tfidf', ...), ('logreg', ...)]"
,transform_input,None
,memory,None
,verbose,False
,input,'content'
,encoding,'utf-8'
,decode_error,'strict'
,strip_accents,'unicode'
,lowercase,True
,preprocessor,None
,tokenizer,None


### Métriques et Confusion Matrix

In [10]:
y_pred = baseline_clf.predict(X_test)
y_proba = baseline_clf.predict_proba(X_test)[:, 1]

print("Baseline 0 - TF-IDF + Regression logistique")
print(classification_report(y_test, y_pred, target_names=["ham(0)", "spam(1)"]))
print("Confusion matrix:\n", confusion_matrix(y_test, y_pred))

Baseline 0 - TF-IDF + Regression logistique
              precision    recall  f1-score   support

      ham(0)       0.99      0.99      0.99       966
     spam(1)       0.96      0.92      0.94       149

    accuracy                           0.98      1115
   macro avg       0.97      0.96      0.96      1115
weighted avg       0.98      0.98      0.98      1115

Confusion matrix:
 [[960   6]
 [ 12 137]]


In [11]:
cm = confusion_matrix(y_test, y_pred)
cm_df = pd.DataFrame(
    cm,
    index=["True ham (0)", "True spam (1)"],
    columns=["Pred ham (0)", "Pred spam (1)"]
)

fig = px.imshow(
    cm_df,
    text_auto=True,
    aspect="auto",
    title="Confusion Matrix : TF-IDF + Logistic Regression"
)
fig.update_layout(xaxis_title="Predicted label", yaxis_title="True label")
fig.show()

In [12]:
# =========================
# PR-AUC (Average Precision) + courbe Precision-Recall (utile en imbalanced)
# =========================
ap = average_precision_score(y_test, y_proba)
print("PR-AUC / Average Precision:", round(ap, 4))

prec, rec, thr = precision_recall_curve(y_test, y_proba)

import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(go.Scatter(x=rec, y=prec, mode="lines", name="PR curve"))
fig.update_layout(
    title=f"Precision-Recall Curve (PR-AUC={ap:.3f})",
    xaxis_title="Recall (spam=1)",
    yaxis_title="Precision (spam=1)"
)
fig.show()


PR-AUC / Average Precision: 0.9736


# Modèle CNN simple

## Tokenisation

In [13]:
import re
import numpy as np

PAD_ID, UNK_ID = 0, 1

def tok(s):  # tokenization ultra simple
    return re.findall(r"[a-z0-9']+", str(s).lower())

def make_vocab(texts, max_vocab=15000):
    from collections import Counter
    c = Counter()
    for t in texts: c.update(tok(t))
    vocab = {"<PAD>": PAD_ID, "<UNK>": UNK_ID}
    for w, _ in c.most_common(max_vocab - 2):
        vocab[w] = len(vocab)
    return vocab

def encode(texts, vocab, max_len=64):
    X = np.full((len(texts), max_len), PAD_ID, dtype=np.int64)
    for i, t in enumerate(texts):
        ids = [vocab.get(w, UNK_ID) for w in tok(t)][:max_len]
        X[i, :len(ids)] = ids
    return X

MAX_LEN = 64
vocab = make_vocab(X_train, max_vocab=15000)

Xtr = encode(X_train, vocab, max_len=MAX_LEN)
Xte = encode(X_test,  vocab, max_len=MAX_LEN)

## Modèle CNN + train

In [14]:
train_loader = DataLoader(
    TensorDataset(torch.tensor(Xtr), torch.tensor(y_train.values).float()),
    batch_size=64, shuffle=True
)
test_loader = DataLoader(
    TensorDataset(torch.tensor(Xte), torch.tensor(y_test.values).float()),
    batch_size=256, shuffle=False
)

class SimpleCNN(nn.Module):
    def __init__(self, vocab_size, emb=32, filters=64, k=3):
        super().__init__()
        self.emb = nn.Embedding(vocab_size, emb, padding_idx=PAD_ID)
        self.conv = nn.Conv1d(emb, filters, kernel_size=k)
        self.fc = nn.Linear(filters, 1)

    def forward(self, x):
        x = self.emb(x).transpose(1, 2)     # [B, emb, L]
        x = torch.relu(self.conv(x))        # [B, filters, L-k+1]
        x = x.max(dim=2).values             # global max pool -> [B, filters]
        return self.fc(x).squeeze(1)        # logits [B]

model = SimpleCNN(len(vocab)).to(device)
opt = torch.optim.AdamW(model.parameters(), lr=2e-3)
loss_fn = nn.BCEWithLogitsLoss()

EPOCHS = 10
for e in range(1, EPOCHS + 1):
    model.train()
    tot = 0.0
    for xb, yb in train_loader:
        xb, yb = xb.to(device), yb.to(device)
        opt.zero_grad()
        logits = model(xb)
        loss = loss_fn(logits, yb)
        loss.backward()
        opt.step()
        tot += loss.item()
    print(f"Epoch {e}/{EPOCHS} | loss={tot:.3f}", flush=True)


Epoch 1/10 | loss=27.993
Epoch 2/10 | loss=13.717
Epoch 3/10 | loss=6.552
Epoch 4/10 | loss=3.523
Epoch 5/10 | loss=2.027
Epoch 6/10 | loss=1.199
Epoch 7/10 | loss=0.720
Epoch 8/10 | loss=0.453
Epoch 9/10 | loss=0.317
Epoch 10/10 | loss=0.233


## Métriques et Confusion Matrix

In [15]:
model.eval()
probs = []
with torch.no_grad():
    for xb, _ in test_loader:
        xb = xb.to(device)
        p = torch.sigmoid(model(xb)).cpu().numpy()
        probs.append(p)

y_proba_cnn = np.concatenate(probs)
y_pred_cnn = (y_proba_cnn >= 0.5).astype(int)

print("SimpleCNN - Classification report")
print(classification_report(y_test, y_pred_cnn, target_names=["ham(0)", "spam(1)"]))
print("PR-AUC:", round(average_precision_score(y_test, y_proba_cnn), 4))
print("Confusion matrix:\n", confusion_matrix(y_test, y_pred_cnn))


SimpleCNN - Classification report
              precision    recall  f1-score   support

      ham(0)       0.99      0.99      0.99       966
     spam(1)       0.93      0.91      0.92       149

    accuracy                           0.98      1115
   macro avg       0.96      0.95      0.96      1115
weighted avg       0.98      0.98      0.98      1115

PR-AUC: 0.9624
Confusion matrix:
 [[956  10]
 [ 13 136]]


In [16]:
cm = confusion_matrix(y_test, y_pred_cnn)

cm_df = pd.DataFrame(
    cm,
    index=["True ham (0)", "True spam (1)"],
    columns=["Pred ham (0)", "Pred spam (1)"]
)

fig = px.imshow(
    cm_df,
    text_auto=True,
    aspect="auto",
    title="Confusion Matrix : SimpleCNN (PyTorch)"
)
fig.update_layout(xaxis_title="Predicted label", yaxis_title="True label")
fig.show()

# Modèle RNN simple 

## Train

In [17]:
train_loader = DataLoader(
    TensorDataset(torch.tensor(Xtr), torch.tensor(np.asarray(y_train)).float()),
    batch_size=64, shuffle=True
)
test_loader = DataLoader(
    TensorDataset(torch.tensor(Xte), torch.tensor(np.asarray(y_test)).float()),
    batch_size=256, shuffle=False
)

class SimpleGRU(nn.Module):
    def __init__(self, vocab_size, emb=32, hidden=64, pad_id=0):
        super().__init__()
        self.emb = nn.Embedding(vocab_size, emb, padding_idx=pad_id)
        self.gru = nn.GRU(emb, hidden, batch_first=True)
        self.fc = nn.Linear(hidden, 1)

    def forward(self, x):
        x = self.emb(x)          # [B, L, emb]
        _, h = self.gru(x)       # h: [1, B, hidden]
        logits = self.fc(h[-1]).squeeze(1)  # [B]
        return logits

model = SimpleGRU(vocab_size=len(vocab), emb=32, hidden=64, pad_id=PAD_ID).to(device)
opt = torch.optim.AdamW(model.parameters(), lr=2e-3)
loss_fn = nn.BCEWithLogitsLoss()

EPOCHS = 10
for e in range(1, EPOCHS + 1):
    model.train()
    tot = 0.0
    for xb, yb in train_loader:
        xb, yb = xb.to(device), yb.to(device)
        opt.zero_grad()
        logits = model(xb)
        loss = loss_fn(logits, yb)
        loss.backward()
        opt.step()
        tot += loss.item()
    print(f"Epoch {e}/{EPOCHS} | loss={tot:.3f}", flush=True)

Epoch 1/10 | loss=30.745
Epoch 2/10 | loss=25.230
Epoch 3/10 | loss=12.259
Epoch 4/10 | loss=7.604
Epoch 5/10 | loss=5.770
Epoch 6/10 | loss=3.185
Epoch 7/10 | loss=2.396
Epoch 8/10 | loss=1.786
Epoch 9/10 | loss=1.361
Epoch 10/10 | loss=0.977


## Métriques et Confusion Matrix

In [18]:
model.eval()
probs = []
with torch.no_grad():
    for xb, _ in test_loader:
        xb = xb.to(device)
        p = torch.sigmoid(model(xb)).cpu().numpy()
        probs.append(p)

y_proba_rnn = np.concatenate(probs)
y_pred_rnn = (y_proba_rnn >= 0.5).astype(int)

print("SimpleGRU (RNN) — Classification report")
print(classification_report(y_test, y_pred_rnn, target_names=["ham(0)", "spam(1)"]))
print("PR-AUC:", round(average_precision_score(y_test, y_proba_rnn), 4))
print("Confusion matrix:\n", confusion_matrix(y_test, y_pred_rnn))

SimpleGRU (RNN) — Classification report
              precision    recall  f1-score   support

      ham(0)       0.98      0.99      0.99       966
     spam(1)       0.93      0.87      0.90       149

    accuracy                           0.97      1115
   macro avg       0.95      0.93      0.94      1115
weighted avg       0.97      0.97      0.97      1115

PR-AUC: 0.9577
Confusion matrix:
 [[956  10]
 [ 19 130]]


In [19]:
cm = confusion_matrix(y_test, y_pred_rnn)

cm_df = pd.DataFrame(
    cm,
    index=["True ham (0)", "True spam (1)"],
    columns=["Pred ham (0)", "Pred spam (1)"]
)

fig = px.imshow(
    cm_df,
    text_auto=True,
    aspect="auto",
    title="Confusion Matrix : SimpleRNN (PyTorch)"
)
fig.update_layout(xaxis_title="Predicted label", yaxis_title="True label")
fig.show()

# Modèle DistilBERT

https://thesai.org/Downloads/Volume15No11/Paper_129-Optimized_SMS_Spam_Detection.pdf

In [20]:
MODEL_NAME = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

class SMSDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=64):
        self.texts = list(texts)
        self.labels = list(labels) if labels is not None else None
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])

        enc = self.tokenizer(
            text,
            truncation=True,
            padding="max_length",
            max_length=self.max_length,
            return_tensors="pt"
        )

        item = {k: v.squeeze(0) for k, v in enc.items()}  # (seq_len,)
        if self.labels is not None:
            item["labels"] = torch.tensor(int(self.labels[idx]), dtype=torch.long)
        return item


In [21]:
# Split train -> train/val (stratifié)
X_tr, X_val, y_tr, y_val = train_test_split(
    X_train, y_train,
    test_size=0.2,
    stratify=y_train,
    random_state=RANDOM_STATE
)

train_ds = SMSDataset(X_tr,  y_tr,  tokenizer, max_length=128)
val_ds   = SMSDataset(X_val, y_val, tokenizer, max_length=128)
test_ds  = SMSDataset(X_test, y_test, tokenizer, max_length=128)

BATCH_SIZE = 8

train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)
val_loader   = DataLoader(val_ds,   batch_size=BATCH_SIZE, shuffle=False)
test_loader  = DataLoader(test_ds,  batch_size=BATCH_SIZE, shuffle=False)

len(train_ds), len(val_ds), len(test_ds)

(3565, 892, 1115)

In [22]:
model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=2
).to(device)

EPOCHS = 3
LR = 2e-5

optimizer = torch.optim.AdamW(model.parameters(), lr=LR)

total_steps = EPOCHS * len(train_loader)
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=int(0.1 * total_steps),
    num_training_steps=total_steps
)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [23]:
def run_eval(model, loader):
    model.eval()
    all_labels = []
    all_probs = []
    all_preds = []

    with torch.no_grad():
        for batch in loader:
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(**batch)
            logits = outputs.logits  # (bs, 2)

            probs = torch.softmax(logits, dim=1)[:, 1]  # proba spam
            preds = torch.argmax(logits, dim=1)

            all_labels.extend(batch["labels"].detach().cpu().numpy().tolist())
            all_probs.extend(probs.detach().cpu().numpy().tolist())
            all_preds.extend(preds.detach().cpu().numpy().tolist())

    return np.array(all_labels), np.array(all_preds), np.array(all_probs)


def train_one_epoch(model, loader):
    model.train()
    total_loss = 0.0

    for batch in loader:
        batch = {k: v.to(device) for k, v in batch.items()}

        optimizer.zero_grad()
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()

        total_loss += loss.item()

    return total_loss / max(1, len(loader))


In [24]:
best_val_f1 = -1
best_state = None

for epoch in range(1, EPOCHS + 1):
    train_loss = train_one_epoch(model, train_loader)

    yv, pv, probv = run_eval(model, val_loader)
    val_f1 = f1_score(yv, pv, pos_label=1)

    print(f"Epoch {epoch}/{EPOCHS} | train_loss={train_loss:.4f} | val_f1_spam={val_f1:.4f}")

    if val_f1 > best_val_f1:
        best_val_f1 = val_f1
        best_state = {k: v.detach().cpu().clone() for k, v in model.state_dict().items()}

# restore best
model.load_state_dict(best_state)
print("Best val F1 spam:", round(best_val_f1, 4))


Epoch 1/3 | train_loss=0.1320 | val_f1_spam=0.9289
Epoch 2/3 | train_loss=0.0241 | val_f1_spam=0.9573
Epoch 3/3 | train_loss=0.0104 | val_f1_spam=0.9573
Best val F1 spam: 0.9573


## Métriques et Confusion Matrix

In [25]:
yt, pt, probt = run_eval(model, test_loader)

print("DistilBERT (PyTorch) - Classification report")
print(classification_report(yt, pt, target_names=["ham(0)", "spam(1)"]))
print("Confusion matrix:\n", confusion_matrix(yt, pt))

DistilBERT (PyTorch) - Classification report
              precision    recall  f1-score   support

      ham(0)       0.99      1.00      0.99       966
     spam(1)       0.98      0.95      0.97       149

    accuracy                           0.99      1115
   macro avg       0.99      0.97      0.98      1115
weighted avg       0.99      0.99      0.99      1115

Confusion matrix:
 [[963   3]
 [  7 142]]


In [26]:
cm = confusion_matrix(yt, pt)

cm_df = pd.DataFrame(
    cm,
    index=["True ham (0)", "True spam (1)"],
    columns=["Pred ham (0)", "Pred spam (1)"]
)

fig = px.imshow(
    cm_df,
    text_auto=True,
    aspect="auto",
    title="Confusion Matrix : DistilBERT (PyTorch)"
)
fig.update_layout(xaxis_title="Predicted label", yaxis_title="True label")
fig.show()

In [27]:
ap = average_precision_score(yt, probt)
print("PR-AUC / Average Precision:", round(ap, 4))

prec, rec, thr = precision_recall_curve(yt, probt)

import plotly.graph_objects as go
fig = go.Figure()
fig.add_trace(go.Scatter(x=rec, y=prec, mode="lines", name="PR curve"))
fig.update_layout(
    title=f"Precision-Recall Curve — DistilBERT (PR-AUC={ap:.3f})",
    xaxis_title="Recall (spam=1)",
    yaxis_title="Precision (spam=1)"
)
fig.show()


PR-AUC / Average Precision: 0.9887


# Agrégation des résultats

In [28]:


# Synthèse des métriques
dummy_strat_metrics = {
    "model": "Naïf (stratified random)",
    "precision_spam": precision_score(y_test, y_pred_strat, pos_label=1, zero_division=0),
    "recall_spam": recall_score(y_test, y_pred_strat, pos_label=1, zero_division=0),
    "f1_spam": f1_score(y_test, y_pred_strat, pos_label=1, zero_division=0),
    "pr_auc": average_precision_score(y_test, y_proba_strat),
}

baseline_metrics = {
    "model": "TF-IDF + LogReg",
    "precision_spam": precision_score(y_test, y_pred, pos_label=1),
    "recall_spam": recall_score(y_test, y_pred, pos_label=1),
    "f1_spam": f1_score(y_test, y_pred, pos_label=1),
    "pr_auc": average_precision_score(y_test, y_proba),
}

cnn_metrics = {
    "model": "SimpleCNN (PyTorch)",
    "precision_spam": precision_score(y_test, y_pred_cnn, pos_label=1, zero_division=0),
    "recall_spam": recall_score(y_test, y_pred_cnn, pos_label=1, zero_division=0),
    "f1_spam": f1_score(y_test, y_pred_cnn, pos_label=1, zero_division=0),
    "pr_auc": average_precision_score(y_test, y_proba_cnn),
}

rnn_metrics = {
    "model": "SimpleGRU (PyTorch)",
    "precision_spam": precision_score(y_test, y_pred_rnn, pos_label=1, zero_division=0),
    "recall_spam": recall_score(y_test, y_pred_rnn, pos_label=1, zero_division=0),
    "f1_spam": f1_score(y_test, y_pred_rnn, pos_label=1, zero_division=0),
    "pr_auc": average_precision_score(y_test, y_proba_rnn),
}

bert_metrics = {
    "model": "DistilBERT (PyTorch)",
    "precision_spam": precision_score(yt, pt, pos_label=1),
    "recall_spam": recall_score(yt, pt, pos_label=1),
    "f1_spam": f1_score(yt, pt, pos_label=1),
    "pr_auc": average_precision_score(yt, probt),
}

pd.DataFrame([dummy_strat_metrics, baseline_metrics, cnn_metrics, rnn_metrics, bert_metrics]).round(4)

,model,precision_spam,recall_spam,f1_spam,pr_auc
0,Naïf (stratified random),0.0970,0.1074,0.1019,0.1297
1,TF-IDF + LogReg,0.9580,0.9195,0.9384,0.9736
2,SimpleCNN (PyTorch),0.9315,0.9128,0.9220,0.9624
3,SimpleGRU (PyTorch),0.9286,0.8725,0.8997,0.9577
4,DistilBERT (PyTorch),0.9793,0.9530,0.9660,0.9887
